<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz

Cloning into 'StockSellRulesChecker'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 72 (delta 21), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (72/72), done.
     |████████████████████████████████| 9.5MB 6.1MB/s 
     |████████████████████████████████| 1.3MB 36.1MB/s 
     |████████████████████████████████| 143kB 40.6MB/s 
     |████████████████████████████████| 296kB 49.5MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=5c4cda0e39f54f96e8eb35f414d9417173b9925bf1a816c7362e782257935dd2
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=e56ec7295e5bf79345770c800f787592a1bc3d94c853e5a76d419b3d850dd31c
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b467

  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-fe9my5li
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-fe9my5li
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=624c16ab25b6706e3a1d23cdeb83d31f1173166b90a14f392274a497e99f8788
  Stored in directory: /tmp/pip-ephem-wheel-cache-e69umfe0/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4


In [2]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time

In [10]:
eps_5year_over5pct = False
eps_qoq_over5pct = False
eps_yoy_over5pct = False
sales_5years_over5pct = False
sales_qoq_over5pct = False

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [11]:
print(len(stock_list.data))

7940


In [12]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list, total=len(stock_list.data)):
  print(stock['Ticker'])
  if stock["Ticker"] == "":
    continue
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    continue

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  print(volume, prev_close, volume*prev_close, 15000000)
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False
  screened_stocks[stock['Ticker']]['eps_ttm_rule'] = eps_ttm_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  if SMA200_percent > 0:
      SMA200_slope_rule = True
  else:
      SMA200_slope_rule = False
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


  0%|          | 0/7940 [00:00<?, ?it/s]

GEN


  0%|          | 1/7940 [00:00<1:37:26,  1.36it/s]

2943236.0 0.53 1559915.08 15000000
AAU


  0%|          | 2/7940 [00:01<1:45:51,  1.25it/s]

...

2907.0 3700.0 10755900.0 15000000
NVR


100%|█████████▉| 7939/7940 [4:03:03<00:01,  1.63s/it]

73341.0 4421.85 324302900.85 15000000
BRK-A


100%|██████████| 7940/7940 [4:03:05<00:00,  1.84s/it]

2456.0 383525.0 941937400.0 15000000


In [13]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [14]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule,eps_ttm_rule,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
5833,DAC,20.328561,24.863558,38.951111,253.03,50.65,16.1,6.52,8.5,49.18,53.580002,2.780000,0.293,20640000.0,681474.0,True,True,True,True,True,True,True,True,True,True,True,True,12.0
7177,DIT,90.944317,99.531539,113.964166,49.61,4.28,600.5,14.57,12.4,117.27,133.910004,52.400002,0.166,550000.0,1166.0,False,True,True,True,True,True,True,True,True,True,True,True,11.0
3577,ESCA,20.579712,21.176635,21.837500,20.30,3.85,98.5,1.82,59.1,22.23,24.049999,4.750000,0.307,14030000.0,48806.0,False,True,True,True,True,True,True,True,True,True,True,True,11.0
7188,JACK,91.585755,94.458173,101.472778,34.44,16.46,569.4,5.75,10.0,114.85,116.830002,23.570000,0.000,22970000.0,831587.0,True,True,True,True,True,False,True,True,True,True,True,True,11.0
7181,DOOR,100.381151,101.540673,110.780000,23.75,8.50,1580.5,2.75,16.4,116.00,119.660004,36.759998,0.000,24450000.0,350133.0,True,True,True,True,True,False,True,True,True,True,True,True,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5923,SNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,0.0
1141,GNSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,0.0
5418,FEUZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,0.0
3242,GFED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,0.0


In [1]:
df_out[df_out["Passed Tests"] >= 11]

NameError: ignored